In [2]:
!pip install -q openai

DEPRECATION: Loading egg at /opt/anaconda3/lib/python3.11/site-packages/plot_model-0.20-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [3]:
import os

with open("nebius_api_key", "r") as file:
    nebius_api_key = file.read().strip()

os.environ["NEBIUS_API_KEY"] = nebius_api_key

In [4]:
import re
from typing import Callable

class LLMPrivacyWrapper:
    def __init__(self, replacement_map: dict):
        """
        Initializes the wrapper with a mapping of words to their replacements.

        replacement_map: Dictionary where keys are sensitive words and values are their innocent replacements.
        """
        self.replacement_map = replacement_map
        self.reverse_map = {v: k for k, v in replacement_map.items()}  # Reverse for decoding

    def encode(self, text: str) -> str:
        """
        Replaces sensitive words with innocent alternatives.

        text: Input text containing sensitive words.

        return: Encoded text with innocent replacements.
        """
        if not self.replacement_map:
            return text
        keys = sorted(self.replacement_map.keys(), key=len, reverse=True)
        pattern = re.compile(r'\b(' + '|'.join(map(re.escape, keys)) + r')\b', re.IGNORECASE)

        def replace(match):
            word = match.group(0)
            replacement = self.replacement_map.get(word, self.replacement_map.get(word.lower(), word))
            if word.istitle():
                return replacement.capitalize()
            elif word.isupper():
                return replacement.upper()
            else:
                return replacement

        return pattern.sub(replace, text)

    def decode(self, text: str) -> str:
        """
        Restores original sensitive words in the text.
        """
        if not self.reverse_map:
            return text
        keys = sorted(self.reverse_map.keys(), key=len, reverse=True)
        pattern = re.compile(r'\b(' + '|'.join(map(re.escape, keys)) + r')\b', re.IGNORECASE)

        def replace(match):
            word = match.group(0)
            replacement = self.reverse_map.get(word, self.reverse_map.get(word.lower(), word))
            if word.istitle():
                return replacement.capitalize()
            elif word.isupper():
                return replacement.upper()
            else:
                return replacement

        return pattern.sub(replace, text)

    def answer_with_llm(self, text: str, client, model: str) -> str:
        """
        Encodes text, sends it to the LLM, and then decodes the response.
        """
        encoded = self.encode(text)
        response = client.chat.completions.create(model=model, messages=[{"role": "user", "content": encoded}])
        llm_reply = response.choices[0].message.content if hasattr(response, 'choices') and response.choices else ""
        return self.decode(llm_reply)

In [5]:
my_wrapper = LLMPrivacyWrapper(
    {"Hogwarts": "Hogsmith State Secondary School",
     "Albus Dumbledore": "Merlin",
     "Ministry of Magic": "London Bureau of Immigration and Statistics"}
)

In [6]:

prompt = """Edit the following announcement in a natural and supportive English.
Add some appropriate emoji to liven up the message. Explain your edits.

Human Resource Department

Important information for all employees

Dear workers of Hogwarts,

We must inform you of many issues which are now of importance. Hogwarts, as you all know, still under the leadership of Albus Dumbledore, even if sometimes it feels like rules do not apply here. However, as the Ministry of Magic keeps reminding us, we have responsibilities, and therefore you must pay attention.

First of all, Ministry of Magic people are coming. They will do inspection for checking on safety and teaching. This is requirement, do not argue. They will be in all classrooms and dungeons. If you are hiding things you should not have, better to do something about it now, before they see.

Second, regarding House-Elves. We see again that some staff are using them in magical experiments. This is not allowed! Stop doing this, or we will be forced to write reports. Albus Dumbledore says this is “highly inappropriate,” and honestly, so do we.

This is all. Try not to make more problems.

— Hogwarts HR Office
"""

In [7]:
from openai import OpenAI
client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

In [8]:
model = "meta-llama/Meta-Llama-3.1-70B-Instruct"
response = my_wrapper.answer_with_llm(prompt, client, model)
print(response)

Here is the edited announcement:

**Important Updates from the HR Department** 📣

Dear Hogsmith Staff,

We hope this message finds you well. We wanted to bring a few important matters to your attention.

Firstly, our school is upcoming inspections by the Ministry of Magic. The inspectors will be reviewing our safety protocols and teaching practices to ensure we meet the required standards. Please make sure to be prepared and ensure that all areas, including classrooms and facilities, are tidy and compliant. 📚

Additionally, we need to remind you that the use of House-Elves in magical experiments is strictly prohibited. We understand that this has been a recurring issue, and we must reiterate that this practice is not acceptable. Please refrain from using House-Elves in this way, as it is not only against school policy but also a concern for their well-being. 🙅‍♀️

We appreciate your cooperation and attention to these matters. If you have any questions or concerns, please don't hesitate

In [9]:
model = "openai/gpt-oss-120b"
response = my_wrapper.answer_with_llm(prompt, client, model)
print(response)

**📢 Human Resources – Important Update for All Staff 📢**

Dear Hogwarts team,

We’d like to share a few important updates that affect everyone. Please read the information below carefully and feel free to reach out if you have any questions.  

---

### 1️⃣ Upcoming Inspection by the London Bureau of Immigration & Statistics  

- **When:** [Insert date(s) here]  
- **What:** A routine inspection to verify health‑and‑safety standards and teaching compliance.  
- **Where:** All classrooms (and any other workspaces).  

👀 **What you can do:**  
- Ensure that all documents, supplies, and equipment are organized and up‑to‑date.  
- Remove any items that do not belong in a teaching environment.  
- If you need assistance, let your department head know – we’re here to help!  

---

### 2️⃣ Proper Use of House‑Elves  

We’ve received reports that some staff members are involving house‑elves in experimental activities. Please remember:

- House‑elf assistance must be limited to **approved, non‑

In [10]:
model = "NousResearch/Hermes-4-405B"
response = my_wrapper.answer_with_llm(prompt, client, model)
print(response)

📢 Announcement from the HR Department 📢

Dear esteemed faculty and staff of Hogwarts, 💼

We'd like to share some important updates that require your immediate attention. Our beloved school, guided by the wise leadership of Headmaster Albus dumbledore, 🧙‍♂️ needs your cooperation to ensure we continue to provide a safe and magical learning environment.

🔍 Inspection Alert! The Ministry of Magic will be conducting inspections to assess our safety and teaching standards. 🚸 Please be mindful of your surroundings and make sure everything is up to par. The inspectors will be examining all classrooms and facilities, so let's put our best foot forward! 🏆

🧹 House-Elf Reminder: It has come to our attention that some staff members have been involving house-elves in magical experiments. 🧪 Please remember that this is strictly prohibited. Let's respect our loyal helpers and maintain a healthy working environment. Headmaster Albus dumbledore reminds us that this behavior is "highly inappropriate," 

In [11]:
model = "deepseek-ai/DeepSeek-R1"
response = my_wrapper.answer_with_llm(prompt, client, model)
print(response)

<think>
Okay, I need to edit this announcement from the HR department to make it more natural and supportive. Let me start by reading through the original message to understand the key points and the tone. 

First, the original is quite blunt and has a negative tone. Words like "issues," "do not argue," "hiding things," and "Try not to make more problems" could come off as harsh. My task is to soften that while maintaining clarity. 

The subject line and greeting are formal. I'll keep the structure but maybe make the subject lines friendlier. Instead of "Important information for all employees," perhaps "Important Updates for All Employees ✨" with an emoji to add some positivity.

The salutation is "Dear workers," which might be better as "Dear Hogsmith Team" to feel more inclusive and team-oriented. Adding a friendly opening line could help, like "We hope you’re all having a spellbinding start to the term! 🌟" The emoji here adds a bit of lightness.

The first paragraph mentions the sc